In [1]:
!pip install tensorflow
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from tabulate import tabulate
from tqdm import trange

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/ISOT/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/ISOT


In [21]:
dfts=pd.read_csv('isottestmeta.csv')
dftr=pd.read_csv('isottrainmeta.csv')
ptr=pd.read_csv('test_base_&_train_meta.csv')
pts=pd.read_csv('test_meta.csv')

In [22]:
ptr=ptr.dropna()
pts=pts.dropna()
ptr['label']=ptr['label'].astype(int)
pts['label']=pts['label'].astype(int)

<ipython-input-22-1ad54358437f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptr['label']=ptr['label'].astype(int)


In [23]:
dfts['cnn']=dfts['cnn'].astype(int)
dftr['cnn']=dftr['cnn'].astype(int)

In [ ]:
print(len(pts))

14336


In [24]:

cols=['bert','cnn','lg','lstm','svm','nb']

In [25]:
import itertools

subsets = []
for r in range(2, len(cols) + 1):
    indices = range(len(cols))
    subset_indices = list(itertools.combinations(indices, r))
    for subset_index in subset_indices:
        subset = [cols[i] for i in subset_index]
        subsets.append(subset)



In [26]:
def evalmeta(dftr,dfts,cols):
  x_train=pd.DataFrame()
  y_train=pd.DataFrame()
  x_test=pd.DataFrame()
  y_test=pd.DataFrame()
  y_train['label']=ptr['label']
  y_test['label']=pts['label']
  for coli in cols:
    x_train[coli]=dftr[coli]
    x_test[coli]=dfts[coli]
  # print(len(x_train))
  # print(len(y_train))
  # print(x_test)
  # print(y_test)
  metamodel=Sequential()
  metamodel.add(Dense(7,input_dim=len(cols),activation='relu'))
  metamodel.add(Dense(1,activation='sigmoid'))
  metamodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  history = metamodel.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=32)
  y_pred = metamodel.predict(x_test)

  vot_pred=np.round(y_pred).flatten()
  accuracy = accuracy_score(list(vot_pred), y_test)
  print(classification_report(list(vot_pred), y_test,digits=4))


In [27]:
for subset in subsets:
    if len(subset) > 1 and len(subset)<4:
      print("Accuracy for Stacking using these classifiers =")
      print(subset)
      
      evalmeta(dftr,dfts,subset)

Accuracy for Stacking using these classifiers =
['bert', 'cnn']
Epoch 1/2
316/316 [==============================] - 2s 5ms/step - loss: 0.5818 - accuracy: 0.7304 - val_loss: 0.4139 - val_accuracy: 0.9978
Epoch 2/2
316/316 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

         0.0     0.9962    0.9996    0.9979      5266
         1.0     0.9996    0.9959    0.9977      4837

    accuracy                         0.9978     10103
   macro avg     0.9979    0.9977    0.9978     10103
weighted avg     0.9978    0.9978    0.9978     10103

Accuracy for Stacking using these classifiers =
['bert', 'lg']
Epoch 1/2
316/316 [==============================] - 2s 4ms/step - loss: 0.4903 - accuracy: 0.9890 - val_loss: 0.3762 - val_accuracy: 0.9898
Epoch 2/2
316/316 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

         0.0     0.9807    0.9998    0.9902      5183
         1.0     0.9998  

In [ ]:
accuracy = accuracy_score(dftr['bert'],ptr['label'])
print(accuracy)

0.9640493038119151


In [ ]:
predictors=df.iloc[:,1:6]
response=df.iloc[:,0]

In [ ]:
# Train the model


Epoch 1/2
225/225 [==============================] - 2s 3ms/step - loss: 0.4912 - accuracy: 0.9833 - val_loss: 0.3301 - val_accuracy: 0.9961
Epoch 2/2
225/225 [==============================] - 1s 4ms/step - loss: 0.2421 - accuracy: 0.9942 - val_loss: 0.1550 - val_accuracy: 0.9967


In [ ]:

resdf = pd.DataFrame(y_pred_classes, columns=['label'])

57/57 [==============================] - 0s 2ms/step


In [ ]:
print("Accuracy with Meta Model: {}".format(accuracy_score(y_test, y_pred_classes)))

Accuracy with Meta Model: 0.9966592427616926
